# Display of Ear aqueduct results (finite difference code - diffusion only)

Importing the libraries

In [ ]:
import numpy as np
import ipyplot

#### Create a list of images that live in `../../Collab-BrainEfflux-Data/April_2x_2024`
find all files in the directory of the form *.png

In [ ]:
#case =  "April_2x_2024_b"
#case =  "v06May2024_a"
#case = "v21May2024_c"
#case = "v20May2024_const_b"
# case = "v16Aug2024_real"
case = "v16Aug2024_synth_large_a"
#case = "v16Aug2024_synth_small_a"


if case == "April_2x_2024_b":
    images = !find ../../../Collab-BrainEfflux-Data/April_2x_2024_b/ -name "*.png"
    noise_idx= -8

if case == "v06May2024_a":
    images = !find ../../../Collab-BrainEfflux-Data/v06May2024_a/ -name "*New*.png"
    noise_idx= -6

if case == "v21May2024_c":
    images = !find ../../../Collab-BrainEfflux-Data/v21May2024_c/ -name "*.png"
    noise_idx= -6

if case == "v20May2024_const_b":
    images = !find ../../../Collab-BrainEfflux-Data/v20May2024_const_b/ -name "*.png"
    noise_idx= -6


if case == "v16Aug2024_real":
    images = !find ../../../Collab-BrainEfflux-Data/v16Aug2024_real/ -name "*.png"
    noise_idx= -6
    num_samples_labels_offset = 2
    
if case == "v16Aug2024_synth_large_a":
    images = !find ../../../Collab-BrainEfflux-Data/v16Aug2024_synth_large_a/ -name "*.png"
    num_samples_labels_offset = 2
    noise_idx= -8

if case == "v16Aug2024_synth_small_a":
    images = !find ../../../Collab-BrainEfflux-Data/v16Aug2024_synth_small_a/ -name "*.png"
    num_samples_labels_offset = 2
    noise_idx= -8


images = np.array(images)


Total number of images:

In [ ]:
len(images)
print(images[0])


#### Create a list of labels for the images

In [ ]:
animal_labels = np.array([image.split('/')[-1].split('_')[1] for image in images])
ear_labels = np.array([image.split('/')[-1].split('_')[2] for image in images])
sampler_labels = np.array([image.split('/')[-1].split('_')[3] for image in images])
data_type_labels = np.array([image.split('/')[-1].split('_')[6] for image in images])
num_samples_labels = np.array([image.split('/')[-1].split('_')[8+num_samples_labels_offset] for image in images])
CA_labels = np.array([image.split('/')[-1].split('_')[-1].split('.')[0] for image in images])
ST_labels = np.array([image.split('/')[-1].split('_')[-2] for image in images])
noise_labels = np.array([image.split('/')[-1].split('_')[noise_idx] for image in images])

# Additional label
animal_ear_labels = np.array([animal_labels[i] + '_' + ear_labels[i] for i in range(len(images))])


Image titles:

In [ ]:
image_titles = np.array([f"{animal_labels[i]}_{ear_labels[i]}_{sampler_labels[i]}_{data_type_labels[i]}_{num_samples_labels[i]}_{CA_labels[i]}_{ST_labels[i]}_{noise_labels[i]}" for i in range(len(images))])

Print all unique labels

In [ ]:
print('animal_labels:', np.unique(animal_labels))
print('ear_labels:', np.unique(ear_labels))
print('sampler_labels:', np.unique(sampler_labels))
print('data_type_labels:', np.unique(data_type_labels))
print('num_samples_labels:', np.unique(num_samples_labels))
print('CA_labels:', np.unique(CA_labels))
print('ST_labels:', np.unique(ST_labels))
print('noise_labels:', np.unique(noise_labels))


#### Filter images based on labels (create idx of images to keep)

In [ ]:
# Initialize idx
idx = np.arange(len(images))

# remove images where number of samples is 20
#idx = np.where(np.logical_and(np.array(num_samples_labels) != '20',
#               np.array(num_samples_labels) != '200' ,
#               np.array(num_samples_labels) != '200000'))[0] 

# Keep images where sampler is NUTS
idx = np.intersect1d(idx, np.where(np.array(sampler_labels) == 'NUTS')[0])

# keep only images where noise is 0.1
# idx = np.intersect1d(idx, np.where(np.array(noise_labels) == 'var')[0])

# keep only images where number of ST measurements is 4
#idx = np.intersect1d(idx, np.where(np.array(ST_labels) == '4')[0])

# keep only animal m1
idx = np.intersect1d(idx, np.where(np.array(animal_labels) == 'm1')[0])

print(len(idx))

#### Display the images

In [ ]:
#classification = animal_ear_labels[idx]
classification = noise_labels[idx]
ipyplot.plot_class_tabs(images[idx],
                        classification,
                        max_imgs_per_tab=10, img_width=400, show_url=False,
                        custom_texts=image_titles[idx])

#### From case April to case May 6: The following changes:
Updates about the prior:
- Prior choice was: `GMRF(np.ones(G_c.par_dim), 2, geometry=G_c, bc_type='neumann')`
- New prior choice: `GMRF(np.ones(G_c.par_dim)*np.sqrt(300), 0.2, geometry=G_c, bc_type='neumann')`

Update about the noise level:
- (This is still being looked at) it seems all noise level set up were probably correct except for the case when we average the noise std. Now it is corrected.
- Do we need to take into account the subsampling of the data?
- noise over time added

May 6 experiment is synthetic